In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/MASTER_AI/

AttentionMil_pretrained1.ipynb	Code			  Pretrainedcontrastive
CAMELYON16			preprocesseddata	  training_image_data
clustering.ipynb		preprocesseddata_highmag  WSI_1_CAMELYON


In [ ]:
path = '/content/gdrive/MyDrive/MASTER_AI/Pretrainedcontrastive'
print(os.path.exists(path))

True


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision

import matplotlib.pyplot as plt

import pytorch_lightning
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
import cv2


import numpy as np

from torch import nn

# import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
"""Pytorch dataset object that loads MNIST dataset as bags."""

class HistoBags(Dataset):
    def __init__(self, seed=1, train=True):
        self.train = train
        self.r = np.random.RandomState(seed)

        if self.train:
            self.train_bags_list, self.train_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _load_images_from_folder(self,folder):
        images = []
        count = 0
        for filename in os.listdir(folder):
            if count >= 1000:
                break
            img = cv2.imread(
                os.path.join(folder, filename))  # by default OpenCV reads image in the sequence Blue Green Red (BGR
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img/255
            if img is not None:
                images.append(img)
            count += 1

        images = np.stack(images, axis=0)
        images = torch.from_numpy(images).float()
        images = images.permute(0, 3, 1, 2)
        return images

    #creates a bag and label list out of the out of normalised MNIST images
    def _create_bags(self):
        bags_list = []
        labels_list = []

        if self.train:
            # normal train images
            # camelyondir = '/Users/imac/Documents/Documents_Pieter_MacBook_Pro/0MasterAI/THESIS/data/CAMELYON/processed_normal_00'
            camelyondir = '/content/gdrive/MyDrive/MASTER_AI/preprocesseddata_highmag/train/normal_'

            for i in range(1, 8):
                path = camelyondir + str(i)
                print(os.path.exists(path))
                images = self._load_images_from_folder(path) # [450,3,224,224] torch.float32
                bags_list.append(images)  # [ [20,3,224,224],  [45,3,224,224], [70,3,224,224],... ]
                labels_list.append(0)


            # tumor train images
            # camelyondir = '/Users/imac/Documents/Documents_Pieter_MacBook_Pro/0MasterAI/THESIS/data/CAMELYON/processed_tumor_00'
            camelyondir = '/content/gdrive/MyDrive/MASTER_AI/preprocesseddata_highmag/train/tumor_'
            for i in range(1, 6):
                path = camelyondir + str(i)
                print(os.path.exists(path))
                images = self._load_images_from_folder(path)
                bags_list.append(images)
                labels_list.append(torch.tensor([1]))
        else:
            # tumor test images
            camelyondir = '/Users/pieterdujardin/Documents/Documents_Pieter_MacBook_Pro/0MasterAI/THESIS/data/CAMELYON/processed_tumor_00'
            for i in range(2, 3):
                path = camelyondir + str(i)
                print(os.path.exists(path))
                images = self._load_images_from_folder(path)
                bags_list.append(images)
                labels_list.append(torch.tensor([1]))

        return bags_list, torch.FloatTensor(labels_list) # bags_list is a list of tensors, labelslist is a tensor of values

    #returns lenght of train or test set
    def __len__(self):
        if self.train:
            return len(self.train_labels_list)
        else:
            return len(self.test_labels_list)

    #returns a certain bag and its label
    def __getitem__(self, index):
        if self.train:
            bag = self.train_bags_list[index]
            label = self.train_labels_list[index]
        else:
            bag = self.test_bags_list[index]
            label = self.test_labels_list[index]

        return bag, label

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
MODEL_PATH = '/content/gdrive/MyDrive/MASTER_AI/Pretrainedcontrastive/tenpercent_resnet18.ckpt'
RETURN_PREACTIVATION = True  # return features from the model, if false return classification logits
NUM_CLASSES = 4  # only used if RETURN_PREACTIVATION = False


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def load_model_weights(model, weights):

    model_dict = model.state_dict()
    weights = {k: v for k, v in weights.items() if k in model_dict}
    if weights == {}:
        print('No weight could be loaded..')
    model_dict.update(weights)
    model.load_state_dict(model_dict)

    return model


pretrainedmodel = torchvision.models.__dict__['resnet18'](pretrained=False)

state = torch.load(MODEL_PATH, map_location='cpu')

state_dict = state['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace('model.', '').replace('resnet.', '')] = state_dict.pop(key)

pretrainedmodel = load_model_weights(pretrainedmodel, state_dict)

if RETURN_PREACTIVATION:
    pretrainedmodel.fc = torch.nn.Sequential()
# else:
#     model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)

pretrainedmodel.cuda()


class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 512
        self.D = 128
        self.K = 1
        #4x1000 and 512x128)

       

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 1),
            nn.Sigmoid()
        )

    # X is input and is one bag
    def forward(self, x):
        
#         print(x.shape)
        x = x.squeeze(0) #remove first dimension the bag tensor
#         print(x.shape)
        
        pretrainedmodel.cuda()
        H = pretrainedmodel(x)
#         print(H.shape) 
        
        # aggregation part
        A = self.attention(H)  # NxK
        # print(A.shape) #torch.Size([11, 1])
        A = torch.transpose(A, 1, 0)  # KxN
        # print(A.shape) #torch.Size([1, 11])
        A = F.softmax(A, dim=1)  # softmax over N
        # print(A.shape) #torch.Size([1, 11])

        # H gets multiplied with A, where A is some kind of multiplied H
        M = torch.mm(A, H)  # KxL #so KxL is the feature of the bag
        # print(M.shape) #torch.Size([1, 500])
        # print(M)

        # final transformation part
        Y_prob = self.classifier(M)  # KxL to a one dim output for probability bag label
        # print(Y_prob.shape, Y_prob) #torch.Size([1, 1])
#         Y_hat = torch.ge(Y_prob, 0.5).float()
        # print('end')
        # print(Y_hat,Y_prob)
#         print(Y_prob)

        return Y_prob #, Y_hat, A

In [ ]:
###########

print('Load Train set')
train_loader = DataLoader(HistoBags(seed=1,train=True),
                                         batch_size=1,
                                         shuffle=True)

# test_loader = DataLoader(HistoBags(seed=1, train=False),
#                          batch_size=1,
#                          shuffle=False,**loader_kwargs)


lr = 0.0003
weight_decay = 0.00
epochs = 100

print('Init Model')
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay= weight_decay)

# print(model)
# summary(model,[10, 50, 4, 4]) #model , Example input tensor of the model  #torch.Size([1, 7, 1, 28, 28]) #can do same by printing torch size after each layer

# model.to(dev)
# data = data.to(dev)


def train(epoch):
    model.cuda()
    model.train() #puts model in training mode
    
    train_loss = 0.
    train_error = 0.
    
    for batch_idx, (data, label) in enumerate(train_loader):    # data is torch.float32 of  torch.Size([1, 21, 3, 224, 224]) and label is torch.float32 and torch.Size([1])
        
        optimizer.zero_grad()
        criterion = nn.BCELoss()
        data, label = data.cuda(), label.cuda()

        output = model(data) #data.float()
        print(output.squeeze(0))
        print(label)
        loss = criterion(output.squeeze(0),label)
        print(loss)
        train_loss += loss.item()
        
        print(train_loss)
        
        
         # backward pass
        loss.backward()
        # step
        optimizer.step()
        
    
    # calculate loss and error for epoch
    train_loss /= len(train_loader)
    # train_error /= len(train_loader)

    print('Epoch: {}, Loss: {:.4f}'.format(epoch, train_loss))


def test():
    model.eval()
    test_loss = 0.
    test_error = 0.
    accuracy = 0
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        instance_labels = label[1]
        if args.cuda:
            data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, attention_weights = model.calculate_objective(data, bag_label)
        test_loss += loss.data[0]
        error, predicted_label = model.calculate_classification_error(data, bag_label)
        test_error += error

        if batch_idx < 20:  # plot bag labels and instance labels for first 5 bags
            bag_level = (bag_label.cpu().data.numpy()[0], int(predicted_label.cpu().data.numpy()[0][0]))
            instance_level = list(zip(instance_labels.numpy()[0].tolist(),
                                 np.round(attention_weights.cpu().data.numpy()[0], decimals=3).tolist()))

            print('\nTrue Bag Label, Predicted Bag Label: {}\n'
                  'True Instance Labels, Attention Weights: {}'.format(bag_level, instance_level))

    test_error /= len(test_loader)
    test_loss /= len(test_loader)
    accuracy = 1- test_error


    print('\nTest Set, Loss: {:.4f}, Test error: {:.4f}'.format(test_loss.cpu().numpy()[0], test_error))
    print('\nTest accuracy: {}'.format(accuracy))
#
if __name__ == "__main__":
    print('Start Training')
    for epoch in range(1, epochs + 1):
        train(epoch)
#     # print('Start Testing')
#     # test()


Load Train set
True
True
True
True
True
True
True
True
True
True
True
True
Init Model
Start Training
tensor([0.4999], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.], device='cuda:0')
tensor(0.6929, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
0.6928690075874329
tensor([0.4923], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([1.], device='cuda:0')
tensor(0.7087, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
1.4015701413154602
tensor([0.4932], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([1.], device='cuda:0')
tensor(0.7068, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
2.108344554901123
tensor([0.4952], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.], device='cuda:0')
tensor(0.6836, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
2.791894853115082
tensor([0.4972], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.], device='cuda:0')
tensor(0.6875, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
3.4794149994850

KeyboardInterrupt: ignored

NameError: ignored

In [ ]:
import torch

In [ ]:
b = torch.rand([270,10000,512])